# logger

In [1]:
import warnings
warnings.filterwarnings('ignore')

import logging

# 로거 생성 및 설정
logger = logging.getLogger()  # 기본 로거 가져오기
logger.setLevel(logging.INFO)  # 로그 수준 설정

# Jupyter Notebook에 출력될 수 있도록 스트림 핸들러 추가
if not logger.handlers:
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s %(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

# invoke lambda

In [2]:
import boto3
import json

In [3]:
# boto3를 사용하여 Lambda 서비스에 접근할 클라이언트 생성
lambda_client = boto3.client('lambda', region_name='us-east-1')

# 호출할 Lambda 함수의 이름
function_name = 'gs-automl-fargate-init'

2024-11-12 07:30:30,873 - INFO Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [4]:
payload = {
    'cluster': 'automl-fargate-cluster',
    'task_definition': 'automl-fargate-papermill-tabular312-langchain', 
    'table_name': 'automl-regression-experiment',
    'project_hashkey': '2ee07a49',
    'experiment_hashkey': 'c36217bf',
    'dataset_table_name': 'automl-dataset',
    'model_repo_table_name': 'automl-model-repo',
    'username': 'sean@gs.co.kr',
    'task_token': 'dummy',
    'cpu': 1024*2,
    'memory': 1024*4,
    'account_id': '703671896240',
    'ecs_role_name': 'RoleStack-automlcdkprodEcsTaskExecutionRoleD78B1370-B8HLhToU7Ysq',
    'subnet': 'subnet-0c930109e965894d3',  # 실제 서브넷 ID로 대체
    'securityGroup': 'sg-031827c50dde9638f',  # 실제 보안 그룹 ID로 대체
}

In [5]:
# Lambda 함수 호출
response = lambda_client.invoke(
    FunctionName=function_name,
    # InvocationType='Event', # 비동기 호출
    InvocationType='RequestResponse', # 동기 호출
    Payload=json.dumps(payload) # event 데이터를 JSON 문자열로 변환
)

print(response)

{'ResponseMetadata': {'RequestId': '21425d48-ef4a-4759-ab3f-ed809e748fd9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 12 Nov 2024 07:30:35 GMT', 'content-type': 'application/json', 'content-length': '518', 'connection': 'keep-alive', 'x-amzn-requestid': '21425d48-ef4a-4759-ab3f-ed809e748fd9', 'x-amzn-remapped-content-length': '0', 'x-amz-executed-version': '$LATEST', 'x-amzn-trace-id': 'Root=1-67330417-0d3782c64a70d78b31e77120;Parent=4533cbb323657996;Sampled=0;Lineage=1:79f592ce:0'}, 'RetryAttempts': 0}, 'StatusCode': 200, 'ExecutedVersion': '$LATEST', 'Payload': <botocore.response.StreamingBody object at 0x7f209b431a50>}


In [6]:
# 응답에서 페이로드 추출 및 디코딩
response_payload = response['Payload'].read()
result = json.loads(response_payload)

print(result)

{'ps': 1731396633, 'task': 'fargate_init', 'created_dt': '2024-11-12 16:30:33', 'table_name': 'automl-regression-experiment', 'project_hashkey': '2ee07a49', 'experiment_hashkey': 'c36217bf', 'dataset_table_name': 'automl-dataset', 'model_repo_table_name': 'automl-model-repo', 'username': 'sean@gs.co.kr', 'task_id': '21d2a564b2ad4fc7b84dd7f829962d8e', 'log_stream_name': 'ecs/automl-fargate-papermill-tabular312-langchain/21d2a564b2ad4fc7b84dd7f829962d8e', 'statusCode': 200, 'body': 'fargate task job is initiated.'}


In [7]:
result['log_stream_name']

'ecs/automl-fargate-papermill-tabular312-langchain/21d2a564b2ad4fc7b84dd7f829962d8e'

# trace log

In [8]:
import boto3
import time

region_name = 'us-east-1'

# ECS, CloudWatch 클라이언트 생성
logs_client = boto3.client('logs')

In [9]:
def tail_task_logs(log_group_name, log_stream_name):
    """
    CloudWatch 로그 그룹에서 특정 ECS Task의 로그를 tail 방식으로 출력하는 함수.
    
    Parameters:
    log_group_name (str): CloudWatch 로그 그룹 이름
    log_stream_name (str): CloudWatch 로그 스트림 이름
    """
    MAX_TIMEOUT = 60 * 10
    ts = time.time() + MAX_TIMEOUT
    next_token = None
    found_wrap_up = False
    retry_delay = 2  # 초기 재시도 대기 시간 (초)
    
    while not found_wrap_up:
        try:
            # 로그 가져오기
            if next_token:
                response = logs_client.get_log_events(
                    logGroupName=log_group_name,
                    logStreamName=log_stream_name,
                    nextToken=next_token,
                    startFromHead=False
                )
            else:
                response = logs_client.get_log_events(
                    logGroupName=log_group_name,
                    logStreamName=log_stream_name,
                    startFromHead=True
                )

            # 로그 이벤트 출력
            for event in response['events']:
                logger.info(event['message'])
                if 'wrap-up' in event['message']:
                    found_wrap_up = True
                    logger.info('Found "wrap-up".')
                    break
            
            if time.time() > ts:
                logger.info('Timeout reached without finding "wrap-up".')
                break
            
            # 다음 토큰 설정
            next_token = response['nextForwardToken']
            
            # 다음 로그 가져오기 전 대기
            time.sleep(2)
            retry_delay = 2  # 재시도 대기 시간 초기화

        except logs_client.exceptions.ResourceNotFoundException:
            if time.time() > ts:
                logger.error("Timeout reached while waiting for the log stream.")
                break
            logger.info("Log stream not found, waiting for provisioning...")
            time.sleep(retry_delay)
            retry_delay = min(retry_delay * 2, 30)  # 지수 백오프, 최대 30초
        except Exception as e:
            logger.error(f"Error while retrieving logs: {str(e)}")
            break

    return found_wrap_up

In [10]:
log_group_name = '/ecs/automl-fargate-papermill-tabular312-langchain'  # 실제 로그 그룹 이름으로 대체
log_stream_name = result['log_stream_name']

In [11]:
tailed = tail_task_logs(log_group_name, log_stream_name)

2024-11-12 07:30:35,251 - INFO Log stream not found, waiting for provisioning...
2024-11-12 07:30:37,400 - INFO Log stream not found, waiting for provisioning...
2024-11-12 07:30:41,492 - INFO Log stream not found, waiting for provisioning...
2024-11-12 07:32:14,105 - INFO 2024-11-12 07:32:07,359 - INFO
2024-11-12 07:32:14,106 - INFO # initialized ------------------
2024-11-12 07:32:14,107 - INFO 2024-11-12 07:32:07,359 - INFO
2024-11-12 07:32:14,109 - INFO # get items from ddb ------------------
2024-11-12 07:32:14,110 - INFO Success : automl-logs table에 item이 저장되었습니다.
2024-11-12 07:32:14,111 - INFO 2024-11-12 07:32:07,580 - INFO
2024-11-12 07:32:14,112 - INFO {   'created_dt': '2024-11-12 16:32:07',
2024-11-12 07:32:14,113 - INFO     'dataset': {   'bucket_name': 'gs-automl-data-profile',
2024-11-12 07:32:14,113 - INFO                    'connector_name': 'upload_csv',
2024-11-12 07:32:14,114 - INFO                    'created_dt': '2024-11-01 '
2024-11-12 07:32:14,115 - INFO        